# Geração de Copy para Lançamentos com Agentes de IA em Grafo

### Objetivo Principal
Este notebook automatiza a criação de textos de marketing (copy) para o lançamento de um infoproduto. Utilizando um briefing detalhado como ponto de partida, uma rede de agentes de IA, orquestrada com **LangGraph**, gera de forma colaborativa e iterativa os principais ativos de comunicação necessários para uma campanha.

### Arquitetura da Solução Local
A solução é construída sobre uma arquitetura que combina um Large Language Model (LLM) com um framework de orquestração de grafos e uma base de conhecimento vetorial local (RAG).

- **LLM**: **Google Gemini 1.5 Flash**, um modelo rápido e eficiente para geração de texto.
- **Framework de Orquestração**: **LangGraph**, para criar um fluxo de trabalho cíclico e com estado, permitindo que os agentes colaborem e refinem o trabalho uns dos outros.
- **RAG (Retrieval-Augmented Generation)**: Utilizamos o **ChromaDB** para criar uma base de conhecimento vetorial localmente. O briefing do lançamento é processado, dividido em partes (chunks), transformado em vetores pelo modelo de embedding do Google e indexado. Os agentes podem consultar essa base para garantir que a copy gerada seja consistente e alinhada à estratégia.

### Fluxo de Execução com LangGraph
O processo é gerenciado por um grafo de estados que coordena os agentes em um fluxo lógico:

1.  **Configuração e Indexação**: O ambiente é preparado, as dependências são instaladas и o briefing do lançamento é carregado e indexado na base de conhecimento local (ChromaDB).
2.  **Análise Inicial (Paralela)**: Três agentes especializados (`Dores & Promessas`, `Objeções & Quebras`, `Headlines & Ângulos`) analisam o briefing simultaneamente para extrair os insights fundamentais.
3.  **Consolidação de Contexto**: Um nó `Consolidador` reúne as análises iniciais, criando um "super contexto" enriquecido que servirá de base para a criação da copy.
4.  **Geração da Copy**: O agente de `Adaptação por Canais` utiliza o contexto enriquecido para criar as primeiras versões da copy para as diferentes plataformas (Email, Ads, Stories, etc.).
5.  **Ciclo de Revisão e Refinamento**:
    - Um agente `Crítico Revisor` avalia a copy gerada, comparando-a com o briefing original e as melhores práticas de marketing.
    - Se a copy for **"APROVADA"**, o fluxo termina.
    - Se for marcada para **"REFINAR"**, o agente de `Adaptação` recebe o feedback e gera uma nova versão, iniciando um novo ciclo de revisão.
6.  **Saída Final**: Após a aprovação, a versão final da copy é salva em arquivos `JSON` e `Markdown`, pronta para uso.

### Passo 1: Instalação das Dependências

Primeiro, vamos instalar todas as bibliotecas necessárias para o nosso projeto.

In [1]:
!pip install -q -U langchain langgraph langchain_google_genai google-generativeai chromadb langchain-community python-dotenv
print("✅ Dependências instaladas com sucesso!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 72.8 MB/s eta 0:00:0

### Passo 2: Configuração do Ambiente

Nesta etapa, importamos as bibliotecas e configuramos a chave de API do Google, que é necessária para usar o modelo Gemini.

In [2]:
import os
import json
import re
from typing import TypedDict, Optional, Dict, Any, List
from google.colab import userdata

# LangChain e LangGraph
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph, END

# --- PARÂMETROS GLOBAIS ---
# Chave de API do Google
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Chave de API do Google configurada.")
except Exception as e:
    print("❌ Chave de API 'GOOGLE_API_KEY' não encontrada. Por favor, configure-a no Google Colab.")

# Configurações do LLM
GEMINI_MODEL = "gemini-2.5-flash"
TEMPERATURE = 0.7

# Configurações do Grafo de Agentes
MAX_REFINEMENT_ATTEMPTS = 2 # Número máximo de tentativas de refinamento da copy

✅ Chave de API do Google configurada.


### Passo 3: O Briefing - Nossa Fonte de Verdade

Todo o processo começa com um briefing detalhado do produto e da estratégia de lançamento. Este documento JSON é a única fonte de verdade para os nossos agentes de IA.

In [3]:
# Cole o seu briefing em formato JSON aqui
BRIEFING_JSON = r'''
{
  "briefing_lancamento": {
    "infoproduto": {
      "nome": "Mentoria de Desenvolvimento Inteligente",
      "produtor": "Mauricio Issei",
      "preco": 999.97,
      "formato": "Mentoria Individual",
      "descricao": "Mentoria individual para desenvolver arquiteturas de soluções complexas de software utilizando IA."
    },
    "publico_alvo": {
      "demografia": "Empreendedores digitais, desenvolvedores de software e profissionais de tecnologia.",
      "problema_principal": "Dificuldade em arquitetar soluções robustas e escaláveis, falta de um método claro para o desenvolvimento, resultando em baixo faturamento e projetos estagnados.",
      "transformacao_principal": "Capacidade de criar soluções de software de alta qualidade e escaláveis do zero, alcançando faturamento de 6 ou 7 dígitos.",
      "objecoes_comuns": [
        "Não tenho conhecimento técnico suficiente",
        "O preço é muito alto para mim",
        "Não tenho tempo para aplicar o método"
      ]
    },
    "posicionamento": {
      "diferencial_competitivo": "Único método que combina estratégia de arquitetura de soluções de software com o poder do desenvolvimento acelerado por ferramentas de IA.",
      "tom_de_voz": "Autoridade, inspirador e prático.",
      "gatilhos_mentais": [
        "Autoridade",
        "Prova Social",
        "Escassez",
        "Reciprocidade"
      ]
    },
    "estrategia_lancamento": {
      "tipo_lancamento": "Semente",
      "meta_campanha": "Vender 50 unidades e faturar R$ 50.000",
      "datas_chave": {
        "inicio_campanha": "2025-09-15",
        "abertura_carrinho": "2025-09-22",
        "fechamento_carrinho": "2025-09-29"
      },
      "canais": [
        "Email Marketing",
        "Meta Ads",
        "Instagram Stories",
        "YouTube (VSL)"
      ]
    }
  }
}
'''

# Carrega o JSON em um dicionário Python
briefing = json.loads(BRIEFING_JSON)
print("✅ Briefing carregado com sucesso!")

✅ Briefing carregado com sucesso!


### Passo 4: Configuração do RAG com Vetorização Local

Para que nossos agentes possam "consultar" o briefing de forma inteligente, vamos criar uma base de conhecimento vetorial local usando o ChromaDB.

#### 4.1. Preparação e Indexação do Briefing

Primeiro, convertemos o briefing de JSON para um formato de texto limpo. Em seguida, dividimos esse texto em pedaços menores (chunks) e os indexamos no ChromaDB.

In [4]:
def canonicalize_briefing_to_text(briefing_dict: Dict[str, Any]) -> str:
    """Converte o dicionário de briefing em um texto estruturado."""
    b = briefing_dict.get("briefing_lancamento", {})
    inf = b.get("infoproduto", {})
    pub = b.get("publico_alvo", {})
    pos = b.get("posicionamento", {})
    est = b.get("estrategia_lancamento", {})
    datas = est.get("datas_chave", {})

    linhas = [
        "# Briefing de Lançamento — Canonicalizado",
        "## Produto",
        f"Nome: {inf.get('nome','')} | Produtor: {inf.get('produtor','')} | Preço: {inf.get('preco','')} | Formato: {inf.get('formato','')}",
        f"Descrição: {inf.get('descricao','')}",
        "\n## Público-alvo & Persona",
        f"Demografia/Psicografia: {pub.get('demografia','')}",
        f"Dor principal: {pub.get('problema_principal','')}",
        f"Transformação: {pub.get('transformacao_principal','')}",
    ]
    if pub.get("objecoes_comuns"):
        linhas.append("Objeções comuns:")
        for o in pub["objecoes_comuns"]:
            linhas.append(f"- {o}")

    linhas.extend([
        "\n## Posicionamento & Diferencial",
        f"USP: {pos.get('diferencial_competitivo','')}",
        f"Tom de voz: {pos.get('tom_de_voz','')}",
    ])
    if pos.get("gatilhos_mentais"):
        linhas.append("Gatilhos prioritários: " + ", ".join(pos["gatilhos_mentais"]))

    linhas.extend([
        "\n## Estratégia de Lançamento",
        f"Tipo: {est.get('tipo_lancamento','')} | Meta: {est.get('meta_campanha','')}",
        f"Período/Datas: início={datas.get('inicio_campanha','')} | abertura={datas.get('abertura_carrinho','')} | fechamento={datas.get('fechamento_carrinho','')}",
    ])
    if est.get("canais"):
        linhas.append("Canais: " + ", ".join(est["canais"]))

    return "\n".join(linhas)

# 1. Converter o briefing para texto
canonical_text = canonicalize_briefing_to_text(briefing)

# 2. Configurar o modelo de embedding e o ChromaDB
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

# 3. Dividir o texto em chunks e indexar
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.create_documents([canonical_text])
vectorstore.add_documents(docs)

print(f"✅ Briefing indexado localmente em {len(docs)} chunks no ChromaDB.")

/tmp/ipython-input-1372927057.py:48: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)


✅ Briefing indexado localmente em 2 chunks no ChromaDB.


#### 4.2. Criação do Retriever

O retriever é o componente que efetivamente busca as informações na nossa base vetorial.

In [5]:
# Cria um retriever a partir do vector store
retriever = vectorstore.as_retriever()

def join_docs(docs: List[Document]) -> str:
    """Formata os documentos recuperados em uma string única."""
    return "\n\n".join(f"[Fonte: doc{i+1}]\n{doc.page_content}" for i, doc in enumerate(docs))

def build_rag_context(briefing_dict: Dict[str, Any]) -> str:
    """Busca informações relevantes no RAG local com base no briefing."""
    pub = briefing_dict.get("briefing_lancamento", {}).get("publico_alvo", {})
    query = pub.get("problema_principal", "briefing do produto") # Usa o problema principal como query

    # Busca os documentos relevantes
    relevant_docs = retriever.get_relevant_documents(query)

    # Formata e retorna o contexto
    return join_docs(relevant_docs)

# Teste rápido do RAG
rag_context = build_rag_context(briefing)
print("--- Contexto RAG Gerado ---")
print(rag_context)
print("---------------------------")

/tmp/ipython-input-891086149.py:14: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(query)


--- Contexto RAG Gerado ---
[Fonte: doc1]
# Briefing de Lançamento — Canonicalizado
## Produto
Nome: Mentoria de Desenvolvimento Inteligente | Produtor: Mauricio Issei | Preço: 999.97 | Formato: Mentoria Individual
Descrição: Mentoria individual para desenvolver arquiteturas de soluções complexas de software utilizando IA.

## Público-alvo & Persona
Demografia/Psicografia: Empreendedores digitais, desenvolvedores de software e profissionais de tecnologia.
Dor principal: Dificuldade em arquitetar soluções robustas e escaláveis, falta de um método claro para o desenvolvimento, resultando em baixo faturamento e projetos estagnados.
Transformação: Capacidade de criar soluções de software de alta qualidade e escaláveis do zero, alcançando faturamento de 6 ou 7 dígitos.
Objeções comuns:
- Não tenho conhecimento técnico suficiente
- O preço é muito alto para mim
- Não tenho tempo para aplicar o método

[Fonte: doc2]
## Posicionamento & Diferencial
USP: Único método que combina estratégia de a

### Passo 5: Construção da Equipe de Agentes de IA

Agora, vamos definir a estrutura e as responsabilidades de cada agente na nossa equipe de IA.

#### 5.1. Definição do Estado Compartilhado

O `AgentState` é um dicionário que funciona como a memória compartilhada entre todos os agentes. Cada agente pode ler e escrever informações neste estado.

In [6]:
class AgentState(TypedDict):
    """Define a estrutura de dados (estado) compartilhada entre os nós do grafo."""
    briefing: Dict
    contexto_rag: str
    dores_promessas: Optional[Dict]
    objecoes_quebras: Optional[Dict]
    headlines_angulos: Optional[Dict]
    contexto_enriquecido: Optional[str]
    copy_por_canal: Optional[Dict]
    revisao_critico: Optional[str]
    tentativas_refinamento: int

print("✅ Estrutura de estado (AgentState) definida.")

✅ Estrutura de estado (AgentState) definida.


#### 5.2. Criação dos Prompts e Cadeias (Cérebro dos Agentes)

Cada agente é movido por um LLM (o Gemini) e um prompt específico que define sua tarefa. A combinação do prompt com o LLM é chamada de "cadeia" (chain).

In [7]:
# Inicializa o LLM que será usado por todos os agentes
llm = ChatGoogleGenerativeAI(model=GEMINI_MODEL, temperature=TEMPERATURE)

# Base do prompt do sistema para todos os agentes
SYSTEM_BASE = (
    "Você é um membro de uma equipe de marketing de elite especialista em copy para lançamentos de infoprodutos. "
    "Sua resposta deve ser sempre um bloco de código JSON, sem nenhum texto adicional antes ou depois. "
    "Utilize o tom de voz definido no briefing. Foque em clareza, estratégia e em gerar ativos prontos para usar. "
    "Respeite a persona (dores, desejos) e o posicionamento (USP) do produto."
)

def force_json(llm_output: Any) -> Dict:
    """Extrai e parseia de forma robusta um bloco JSON da saída do LLM."""
    content_str = llm_output.content if hasattr(llm_output, 'content') else str(llm_output)

    match = re.search(r"```json\s*([\s\S]*?)\s*```", content_str)
    json_str = match.group(1) if match else content_str

    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return {"error": "Falha ao decodificar JSON", "raw_content": json_str}

# --- Cadeia do Agente 1: Dores & Promessas ---
prompt_dores = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Extraia as dores mais profundas do público e as promessas de transformação mais impactantes. Retorne um JSON com chaves 'dores' e 'promessas'."),
    ("human", "Briefing:\n{briefing}\n\nContexto Adicional (RAG):\n{contexto}")
])
chain_dores = prompt_dores | llm

# --- Cadeia do Agente 2: Objeções & Quebras ---
prompt_objecoes = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Liste as objeções mais prováveis do público e crie quebras de objeção persuasivas. Retorne um JSON com chaves 'objecoes' e 'quebras'."),
    ("human", "Briefing:\n{briefing}\n\nContexto Adicional (RAG):\n{contexto}")
])
chain_objecoes = prompt_objecoes | llm

# --- Cadeia do Agente 3: Headlines & Ângulos ---
prompt_headlines = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Crie headlines magnéticas e ângulos de comunicação criativos para os anúncios e emails. Retorne um JSON com chaves 'headlines' e 'angulos'."),
    ("human", "Briefing:\n{briefing}\n\nContexto Adicional (RAG):\n{contexto}")
])
chain_headlines = prompt_headlines | llm

# --- Cadeia do Agente 4: Adaptador de Canais ---
prompt_canais = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Adapte a mensagem para os canais de comunicação, criando a copy final. Se receber um feedback de revisão, aplique as melhorias. Retorne um JSON com chaves 'email', 'stories', 'ads', 'vsl'."),
    ("human", "Briefing Original:\n{briefing}\n\nContexto Enriquecido (Análises dos outros agentes):\n{contexto_enriquecido}\n\nFeedback de Revisão Anterior (se houver):\n{revisao_critico}")
])
chain_canais = prompt_canais | llm

# --- Cadeia do Agente 5: Crítico Revisor ---
prompt_critico = ChatPromptTemplate.from_messages([
    ("system", "Você é um CRÍTICO DE MARKETING sênior e exigente. Sua tarefa é revisar a copy gerada. Se estiver excelente, responda apenas 'APROVADO'. Se precisar de ajustes, responda 'REFINAR:' seguido por uma lista de pontos específicos e acionáveis."),
    ("human", "Briefing Original:\n{briefing}\n\nCopy Gerada para Revisão:\n{copy_por_canal}")
])
chain_critico = prompt_critico | llm

print("✅ Cérebros (prompts e cadeias) de todos os agentes criados.")

✅ Cérebros (prompts e cadeias) de todos os agentes criados.


### Passo 6: Orquestração do Fluxo de Trabalho com LangGraph

Agora, vamos montar o nosso fluxograma (o grafo), definindo como os agentes interagem entre si.

#### 6.1. Definição dos Nós do Grafo

Cada nó representa um agente (ou uma função lógica) que executa uma tarefa.

In [18]:
def node_dores_promessas(state: AgentState) -> Dict[str, Any]:
    print(" Executando nó: Dores & Promessas...")
    result = chain_dores.invoke({"briefing": json.dumps(state['briefing']), "contexto": state['contexto_rag']})
    return {"dores_promessas": force_json(result)}

def node_objecoes_quebras(state: AgentState) -> Dict[str, Any]:
    print(" Executando nó: Objeções & Quebras...")
    result = chain_objecoes.invoke({"briefing": json.dumps(state['briefing']), "contexto": state['contexto_rag']})
    return {"objecoes_quebras": force_json(result)}

def node_headlines_angulos(state: AgentState) -> Dict[str, Any]:
    print(" Executando nó: Headlines & Ângulos...")
    result = chain_headlines.invoke({"briefing": json.dumps(state['briefing']), "contexto": state['contexto_rag']})
    return {"headlines_angulos": force_json(result)}

def node_consolidador(state: AgentState) -> Dict[str, Any]:
    print(" Executando nó: Consolidador de Contexto...")

    # Verificação de erros dos nós anteriores
    for key in ['dores_promessas', 'objecoes_quebras', 'headlines_angulos']:
        if state.get(key) and 'error' in state[key]:
            error_msg = f"Erro detectado no passo '{key}': {state[key]['error']}"
            print(f"❌ {error_msg}")
            # Propaga o erro para interromper o fluxo de forma segura
            return {"contexto_enriquecido": json.dumps({"error": error_msg})}

    contexto_enriquecido = json.dumps({
        "dores_e_promessas": state.get('dores_promessas', {}),
        "objecoes_e_quebras": state.get('objecoes_quebras', {}),
        "headlines_e_angulos": state.get('headlines_angulos', {})
    }, indent=2, ensure_ascii=False)
    return {"contexto_enriquecido": contexto_enriquecido}

def node_adaptacao_canais(state: AgentState) -> Dict[str, Any]:
    print(" Executando nó: Adaptação por Canais...")

    # Verifica se o consolidador encontrou um erro
    contexto = state.get('contexto_enriquecido', '{}')
    if 'error' in json.loads(contexto):
        print("❌ Interrompendo adaptação devido a erro anterior.")
        return {"copy_por_canal": {"error": "Geração interrompida por erro em passo anterior."}}

    tentativas = state.get('tentativas_refinamento', 0) + 1
    revisao = state.get('revisao_critico') or "Nenhuma. Esta é a primeira versão."
    result = chain_canais.invoke({
        "briefing": json.dumps(state['briefing']),
        "contexto_enriquecido": contexto,
        "revisao_critico": revisao
    })
    return {"copy_por_canal": force_json(result), "tentativas_refinamento": tentativas}

def node_critico_revisor(state: AgentState) -> Dict[str, Any]:
    print(" Executando nó: Crítico Revisor...")

    copy_gerada = state.get('copy_por_canal', {})
    if 'error' in copy_gerada:
      print("❌ Interrompendo revisão. A copy não foi gerada corretamente.")
      return {"revisao_critico": "ERRO_NA_GERACAO"}

    result = chain_critico.invoke({
        "briefing": json.dumps(state['briefing']),
        "copy_por_canal": json.dumps(copy_gerada)
    })
    return {"revisao_critico": result.content}

# Nó de decisão para o ciclo de revisão (Lógica aprimorada)
def decidir_pos_critica(state: AgentState) -> str:
    print(" Executando nó: Decisão Pós-Crítica...")
    revisao = state.get('revisao_critico', '')
    tentativas = state.get('tentativas_refinamento', 0)

    if "ERRO_NA_GERACAO" in revisao:
        print(" Decisão: Erro detectado. Finalizando.")
        return "end"
    if "APROVADO" in revisao:
        print(" Decisão: Copy APROVADA. Finalizando.")
        return "end"
    if tentativas >= MAX_REFINEMENT_ATTEMPTS:
        print(f" Decisão: Máximo de {MAX_REFINEMENT_ATTEMPTS} tentativas atingido. Finalizando com a versão atual.")
        return "end" # Termina mesmo que não aprovado para evitar loop infinito
    else:
        print(f" Decisão: Refinar (Tentativa {tentativas + 1}).")
        return "refinar"

print("✅ Nós do grafo definidos com tratamento de erros aprimorado.")

✅ Nós do grafo definidos com tratamento de erros aprimorado.


#### 6.2. Montagem e Compilação do Grafo

Conectamos os nós para definir a ordem e a lógica de execução.

In [9]:
# Montagem do grafo
workflow = StateGraph(AgentState)

# Adiciona os nós
workflow.add_node("analise_dores_promessas", node_dores_promessas)
workflow.add_node("analise_objecoes_quebras", node_objecoes_quebras)
workflow.add_node("analise_headlines_angulos", node_headlines_angulos)
workflow.add_node("consolidador", node_consolidador)
workflow.add_node("adaptacao_canais", node_adaptacao_canais)
workflow.add_node("critico_revisor", node_critico_revisor)

# Define o ponto de entrada e as arestas (conexões)
# A análise inicial ocorre em paralelo
workflow.set_entry_point("analise_dores_promessas")
workflow.set_entry_point("analise_objecoes_quebras")
workflow.set_entry_point("analise_headlines_angulos")

# Após a análise paralela, tudo converge para o consolidador
workflow.add_edge("analise_dores_promessas", "consolidador")
workflow.add_edge("analise_objecoes_quebras", "consolidador")
workflow.add_edge("analise_headlines_angulos", "consolidador")

# Fluxo principal
workflow.add_edge("consolidador", "adaptacao_canais")
workflow.add_edge("adaptacao_canais", "critico_revisor")

# Aresta condicional para o ciclo de revisão
workflow.add_conditional_edges(
    "critico_revisor",
    decidir_pos_critica,
    {
        "refinar": "adaptacao_canais", # Volta para adaptação se precisar refinar
        "end": END # Termina o fluxo se for aprovado
    }
)

# Compila o workflow
app = workflow.compile()
print("✅ Grafo de agentes construído e compilado com sucesso!")

✅ Grafo de agentes construído e compilado com sucesso!


### Passo 7: Execução e Resultados

Com tudo pronto, vamos executar o nosso grafo de agentes e ver a mágica acontecer!

#### 7.1. Execução do Grafo

In [13]:
# Define o estado inicial para a execução
initial_state = AgentState(
    briefing=briefing,
    contexto_rag=rag_context,
    tentativas_refinamento=0
)

print("🚀 Iniciando a execução do grafo de agentes...")

# Invoca o grafo com o estado inicial
# O stream de eventos mostra o progresso em tempo real
final_copy_state = None
for event in app.stream(initial_state):
    # O evento que contém a copy final é a saída do nó 'adaptacao_canais'
    # Vamos capturar o estado completo desse nó assim que ele for processado
    if "adaptacao_canais" in event:
        final_copy_state = event["adaptacao_canais"]

print("\n\n✅ Execução do grafo concluída!")

🚀 Iniciando a execução do grafo de agentes...
 Executando nó: Dores & Promessas...
 Executando nó: Headlines & Ângulos...
 Executando nó: Objeções & Quebras...
 Executando nó: Consolidador de Contexto...
 Executando nó: Adaptação por Canais...
 Executando nó: Crítico Revisor...
 Executando nó: Decisão Pós-Crítica...
 Decisão: Refinar (Tentativa 2).
 Executando nó: Adaptação por Canais...
 Executando nó: Crítico Revisor...
 Executando nó: Decisão Pós-Crítica...
 Decisão: Copy APROVADA. Finalizando.


✅ Execução do grafo concluída!


#### 7.2. Inspeção do Resultado Final

Vamos analisar a copy final gerada pelos agentes.

In [16]:
print(final_copy_state)

{'copy_por_canal': {'email': {'subject': 'CHEGA de Projetos Estagnados: Arquitete Soluções 6 e 7 Dígitos com a IA ao Seu Lado.', 'body': "Olá, empreendedor(a) digital e desenvolvedor(a) de elite,\n\nVocê se sente frustrado(a) por ver seus projetos de software estagnarem, incapazes de escalar, consumindo tempo e recursos sem gerar o faturamento de 6 ou 7 dígitos que você sabe ser possível?\n\nA paralisia constante diante da complexidade, a falta de um método claro para arquitetar soluções robustas, resultando em retrabalho, prazos estourados e a sensação de estar sempre 'apagando incêndios'… Eu sei exatamente como é.\n\nEu sou Mauricio Issei, e ao longo da minha jornada, percebi que o que separa a maioria dos profissionais de tecnologia do sucesso exponencial não é a falta de talento, mas a ausência de uma arquitetura inteligente e o uso estratégico das ferramentas certas.\n\nÉ por isso que criei a **Mentoria de Desenvolvimento Inteligente** – um programa individual e exclusivo para voc

In [19]:
if final_copy_state:
    # Acessa a copy gerada dentro do estado capturado
    final_copy = final_copy_state.get('copy_por_canal', {})

    if 'error' in final_copy:
        print("--- ❌ FALHA NA GERAÇÃO DA COPY ---")
        print(json.dumps(final_copy, indent=2, ensure_ascii=False))
    else:
        print("--- ✅ COPY FINAL GERADA ---")
        print(json.dumps(final_copy, indent=2, ensure_ascii=False))

        # Salvar os resultados em arquivos
        output_dir = "outputs"
        os.makedirs(output_dir, exist_ok=True)

        json_path = os.path.join(output_dir, "copy_final.json")
        md_path = os.path.join(output_dir, "copy_final.md")

        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(final_copy, f, ensure_ascii=False, indent=2)

        with open(md_path, "w", encoding="utf-8") as f:
            f.write("# Assets de Copy Gerados\n\n")
            # Verifica se o conteúdo é um dicionário antes de iterar
            if isinstance(final_copy, dict):
                for canal, conteudo in final_copy.items():
                    f.write(f"## {canal.title()}\n\n")
                    if isinstance(conteudo, dict) or isinstance(conteudo, list):
                        f.write("```json\n")
                        f.write(json.dumps(conteudo, ensure_ascii=False, indent=2))
                        f.write("\n```\n\n")
                    else:
                        f.write(f"{conteudo}\n\n")
            else:
                f.write(str(final_copy))

        print(f"\n✅ Resultados salvos em '{json_path}' e '{md_path}'.")
else:
    print("❌ Nenhuma saída final foi gerada. Verifique os logs de execução do grafo.")

--- ✅ COPY FINAL GERADA ---
{
  "email": {
    "subject": "CHEGA de Projetos Estagnados: Arquitete Soluções 6 e 7 Dígitos com a IA ao Seu Lado.",
    "body": "Olá, empreendedor(a) digital e desenvolvedor(a) de elite,\n\nVocê se sente frustrado(a) por ver seus projetos de software estagnarem, incapazes de escalar, consumindo tempo e recursos sem gerar o faturamento de 6 ou 7 dígitos que você sabe ser possível?\n\nA paralisia constante diante da complexidade, a falta de um método claro para arquitetar soluções robustas, resultando em retrabalho, prazos estourados e a sensação de estar sempre 'apagando incêndios'… Eu sei exatamente como é.\n\nEu sou Mauricio Issei, e ao longo da minha jornada, percebi que o que separa a maioria dos profissionais de tecnologia do sucesso exponencial não é a falta de talento, mas a ausência de uma arquitetura inteligente e o uso estratégico das ferramentas certas.\n\nÉ por isso que criei a **Mentoria de Desenvolvimento Inteligente** – um programa individual